Augment videos

In [ ]:
import cv2
import numpy as np


def augment_image(image, angle):
    height, width = image.shape[:2]
    center = (width/2, height/2)

    rotate_matrix = cv2.getRotationMatrix2D(center, angle, 1)
    return cv2.warpAffine(image, rotate_matrix, (width, height))
    
    # translation_matrix = np.float32([[1, 0, 100], [0, 1, 600]])
    # return cv2.warpAffine(rotated, translation_matrix, (width, height))


def write_aug(vid_name: str, aug_angle: int):
    vid = cv2.VideoCapture(vid_name)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out_name = f"{aug_angle}_{vid_name}"
    out = cv2.VideoWriter(out_name, fourcc, 30.0, (1280, 720))

    while vid.isOpened():
        ret, frame = vid.read()
        if not ret:
            break

        # frame = cv2.bitwise_not(frame)
        frame = augment_image(frame, aug_angle)
        # cv2.imshow("", frame)
        out.write(frame)

        waitkey = cv2.waitKey(1)
        if waitkey == ord('q'):
            break
    
    vid.release()
    out.release()
    cv2.destroyAllWindows()

if __name__ == '__main__':
    for i in ['IMG_1855.MP4', "IMG_1870.MP4"]:
        for j in [45, 135, 180, 225, 315]:
            write_aug(i, j)



Make video predictions

In [ ]:
from ultralytics import YOLO
import cv2 as cv

if __name__ == '__main__':
    VID_NAME = '.mp4' # FILENAME
    SKIP_SEC_BEGINNING = 0

    vid = cv.VideoCapture(VID_NAME)
    vid_fps = vid.get(cv.CAP_PROP_FPS)
    vid.set(1, vid_fps * SKIP_SEC_BEGINNING)
    ret, frame1 = vid.read()
    fr_counter = 0

    model = YOLO(".pt") # FILENAME

    fourcc = cv.VideoWriter_fourcc(*'mp4v')
    out_name = ".mp4" # FILENAME
    out = cv.VideoWriter(out_name, fourcc, 30.0, (640, 512)) # (640, 512) # (1280, 720)
    while True:
        # res = model(cv.cvtColor(frame1, cv.COLOR_BGR2RGB), size=640)
        results = model.predict(source=frame1, conf=0.3)
        for res in results:
            boxes = res.boxes.xyxy
            for box in boxes:
                cv.rectangle(
                    frame1, 
                    (int(box[0]), int(box[1])), 
                    (int(box[2]), int(box[3])), 
                    (0, 0, 255),
                    thickness=1
                    )
                
                # res.save_txt(f"./ann/frame_{str(fr_counter).zfill(6)}.txt")
        
        out.write(frame1)
            
        cv.imshow("", frame1)
        waitkey = cv.waitKey(1)
        if waitkey == ord('q'):
            break

        ret, frame2 = vid.read()
        frame1 = frame2
        fr_counter += 1
        sec = fr_counter / int(vid_fps)

        if not ret or not vid.isOpened():
            break

    out.release()
    vid.release()
    cv.destroyAllWindows()
